In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import os, sys
import pickle
import re
import time
import h5py


from my_packages import signal_elaboration as s

## Find the Coordinates

In [114]:

# def find_all_coordinates(contents=None, directory=False):
#     if directory:
#         contents = os.listdir(directory)
    
#     # using regular expressions split the string where it finds a number between 1 and 5; followed by a number 
#     # between 0 and 9 or nothing; followed by ".csv". This will lea
    
#     reg_expr1 = "[1-9]([0-9]|)\.csv"
#     parsing1 = lambda x: list(filter(None, re.split(reg_expr1, x)))[0]
#     parsing2 = lambda x: x[1:].split("y")
#     parsed_strings = list(map(parsing2, set(map(parsing1, contents))))
    
#     return contents, parsed_strings

In [8]:
# NAS_path = "/NAS"
# folder = 'xfb31/r18'
# directory = os.path.join(NAS_path, folder)
# cont, parsed = find_all_coordinates(directory=directory)
# contents, parsed = find_all_coordinates(contents=cont)

from my_packages.utils import HandlePaths, probes, probes_walk

full_paths = HandlePaths("/NAS")(probes_walk)

print("The paths to measurements are: \n", full_paths)
print("the paths are all accessable: ", all([os.path.exists(p) for p in full_paths]))

The paths to measurements are: 
 ['/NAS/xfb31/r18', '/NAS/XFE_04s/r18', '/NAS/XFE_04s/r18_o', '/NAS/XFR31/r18_o/along_x', '/NAS/XFR31/r18_o/along_y']
the paths are all accessable:  True


In [93]:
class Batch_Generator():
    # this class defines a generator that loads filenames in an array until the numbering
    # returns to 1. Therefore all the measurements of the same point should be grouped in one batch. 
    # The batch generator expects a generator as input, that returns one at a time the filenames. 
    # In particular such a generator is defined as os.scandir(<directory_path>)
    def __init__(self, coordinates):
        self.coordinate_gen = iter(coordinates)
        self.current_batch = []
        self.number_of_batches_done = 0
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        current_coordinates = next(self.coordinate_gen)
        current_name = "x"+current_coordinates[0]+"y"+current_coordinates[1]
        batch = [current_name+str(ii+1)+".csv" for ii in range(50)]

        self.current_batch=batch
        self.number_of_batches_done += 1

        return self.current_batch



class HandleDataDir():
    def __init__(
        self, 
        path = "",
        file_contents = None,
        ):
        self.path = path
        if not file_contents:
            resp = input("type \"y\" to look through the directory contents: ")
            if resp == "y":
                self.file_contents = os.listdir(path)
            else:
                raise Exception("must give the list of contents")
        else:
            self.file_contents=file_contents

    
    @staticmethod
    def _parse_strings(contents):
        reg_expr = "[1-9]([0-9]|)\.csv"

        parsing1 = lambda x: list(filter(None, re.split(reg_expr, x)))[0]
        parsing2 = lambda x: x[1:].split("y")
        parsed_strings = list(map(parsing2, set(map(parsing1, contents))))
        
        return parsed_strings
    
    
    def get_coordinates(self):
        # using regular expressions split the string where it finds a number between 1 and 9; followed by a number 
        # between 0 and 9 or nothing; followed by ".csv". 

        parsed_strings = self._parse_strings(self.file_contents)
        self.parsed_strings = parsed_strings
        coordinates = np.array(parsed_strings, dtype=np.float32)
        x = np.unique(coordinates[:,0]); y = np.unique(coordinates[:,1]) 
        # the coordinate are expressed in micron!
        # transform to mm
        x = x/1e3; y=y/1e3


        # create a grid that can be useful in plotting on the measurement plane. No need to be conscious of memory:
        # the number of points is very small
        grid = np.meshgrid(x,y, indexing="xy", sparse=False, copy=True) 
        self.coordinates = dict(x=x, y=y, grid=grid)

        return x, y, grid
    
    def make_generator(self):
        # get the coordinates from the parsed strings
        self.parsed_strings = self._parse_strings(self.file_contents)

        self.batch_generator = Batch_Generator(self.parsed_strings)

        return self.batch_generator
        
        


In [94]:

hdata=HandleDataDir(full_paths[0], file_contents=fc)

In [95]:
x, y, grid = hdata.get_coordinates()

bgen = hdata.make_generator()

In [97]:
next(bgen)
bgen.number_of_batches_done

2

2

In [42]:
x = cc[:,0]
y = cc[:,1]
print(x.shape, y.shape)

np.unique(x).shape, np.unique(y).shape

(2166,) (2166,)


((57,), (38,))

In [100]:
a=np.array([1,2,3]); b=np.array([4,5])

k, j = np.meshgrid(a,b, sparse=True)




(1, 3)

In [1]:
from my_packages.utils import HandlePaths, probes_walk

probe_paths = HandlePaths()(probes_walk)
print(probe_paths)
    

TypeError: get_paths() takes 2 positional arguments but 3 were given

In [197]:
get_coordinates = GetCoordinates(measurement_points=mp)

In [278]:

get_coordinates.save_measurement_points()

library already exists
table is  {'/NAS/xfb31/r18': 'xfb31'}
path is  /NAS/xfb31/r18
table is  {'/NAS/xfb31/r18': 'xfb31'}
<HDF5 group "/xfb31" (0 members)>


In [204]:
get_coordinates.get_measurement_points(directory)

The directory has not yet been explored!


In [195]:
with h5py.File("measurements.h5", "r") as f:
    print(f.keys())

# with h5py.File("measurements.h5", "r") as f:
#     print(f.keys())
#     print(f["test_group1"])



<KeysViewHDF5 []>


## Base Class for some functionalities

In [248]:
mp

{'/NAS/xfb31/r18': array([[190000.,  72500.],
        [162000.,  84500.],
        [ 92000.,  70500.],
        ...,
        [112000.,  70500.],
        [198000.,  48500.],
        [156000.,  22500.]])}

In [201]:
hf = Hdf5Lib()
hf.available_libs

['xfb31.h5', 'measurements.h5']

## Create a generator with the saved data

In [5]:
## load data

with open("point_coordinates", "rb") as handle:
    coordinates = pickle.load(handle)

    
batch_gen = s.Batch_Generator(coordinates)

In [14]:
next(batch_gen)

['x106000.00y24500.000.csv',
 'x106000.00y24500.001.csv',
 'x106000.00y24500.002.csv',
 'x106000.00y24500.003.csv',
 'x106000.00y24500.004.csv',
 'x106000.00y24500.005.csv',
 'x106000.00y24500.006.csv',
 'x106000.00y24500.007.csv',
 'x106000.00y24500.008.csv',
 'x106000.00y24500.009.csv',
 'x106000.00y24500.0010.csv',
 'x106000.00y24500.0011.csv',
 'x106000.00y24500.0012.csv',
 'x106000.00y24500.0013.csv',
 'x106000.00y24500.0014.csv',
 'x106000.00y24500.0015.csv',
 'x106000.00y24500.0016.csv',
 'x106000.00y24500.0017.csv',
 'x106000.00y24500.0018.csv',
 'x106000.00y24500.0019.csv',
 'x106000.00y24500.0020.csv',
 'x106000.00y24500.0021.csv',
 'x106000.00y24500.0022.csv',
 'x106000.00y24500.0023.csv',
 'x106000.00y24500.0024.csv',
 'x106000.00y24500.0025.csv',
 'x106000.00y24500.0026.csv',
 'x106000.00y24500.0027.csv',
 'x106000.00y24500.0028.csv',
 'x106000.00y24500.0029.csv',
 'x106000.00y24500.0030.csv',
 'x106000.00y24500.0031.csv',
 'x106000.00y24500.0032.csv',
 'x106000.00y24500.0